# Model Experiments — TrilliumWatts

This notebook experiments with LSTM and GRU models using the modular `trillium_watts` package.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
from trillium_watts.config import load_config
from trillium_watts.models.sequences import create_sequences, split_data, fit_scalers, apply_scalers
from trillium_watts.models.training import grid_search, select_best_params, retrain_final_model, evaluate_model
from trillium_watts.models.persistence import save_model
from trillium_watts.visualization.plots_matplotlib import plot_training_curves, plot_predictions_vs_actual

config = load_config()

## 1. Load Processed Data

In [ ]:
df = pd.read_csv(config.data.processed_data_path, index_col=0, parse_dates=True)
features = config.features.feature_columns
target = config.features.target
data = df[features].values
target_index = features.index(target)

window_size = config.model.window_size
X_raw, y_raw = create_sequences(data, window_size, target_index)
X_train_raw, X_test_raw, y_train_raw, y_test_raw = split_data(X_raw, y_raw, config.model.train_split_ratio)
scaler_X, scaler_y = fit_scalers(X_train_raw, y_train_raw)
X_train, y_train = apply_scalers(X_train_raw, y_train_raw, scaler_X, scaler_y)
X_test, y_test = apply_scalers(X_test_raw, y_test_raw, scaler_X, scaler_y)

print(f'Train: {X_train.shape}, Test: {X_test.shape}')

## 2. GRU Grid Search

In [ ]:
results_gru = grid_search('gru', config.model.param_grid, X_train, y_train, X_test, y_test)
best_gru = select_best_params(results_gru)
print(f'Best GRU params: {best_gru["params"]}')
print(f'Best val_mae: {best_gru["val_mae"]:.4f}')
plot_training_curves(best_gru['history'])

## 3. LSTM Grid Search

In [ ]:
results_lstm = grid_search('lstm', config.model.param_grid, X_train, y_train, X_test, y_test)
best_lstm = select_best_params(results_lstm)
print(f'Best LSTM params: {best_lstm["params"]}')
print(f'Best val_mae: {best_lstm["val_mae"]:.4f}')
plot_training_curves(best_lstm['history'])

## 4. Retrain Best Model

In [ ]:
# Pick the best overall
if best_gru['val_mae'] <= best_lstm['val_mae']:
    best_type, best = 'gru', best_gru
else:
    best_type, best = 'lstm', best_lstm

print(f'Best model: {best_type.upper()}')

X_all, y_all = apply_scalers(X_raw, y_raw, scaler_X, scaler_y)
model_final = retrain_final_model(best_type, best, X_all, y_all)

## 5. Evaluate on Test Set

In [ ]:
metrics = evaluate_model(model_final, X_test, y_test, scaler_y)
print(f"MAE:  {metrics['mae']:.2f}")
print(f"RMSE: {metrics['rmse']:.2f}")
print(f"R2:   {metrics['r2']:.4f}")
plot_predictions_vs_actual(metrics['y_true'], metrics['y_pred'])

## 6. Save Model

In [ ]:
save_model(model_final, scaler_X, scaler_y, config.model.model_save_path)
print('Model saved.')